In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats as st
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
#import xesmf as xe
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import cmocean.cm as cmo
import os
import glob
from sys import exit, argv
import netCDF4 as nc4
from distributed import Client
from h5py import File
from pyhdf.SD import SD, SDC
import re
from datetime import datetime


import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader


In [ ]:
data_path = '/Volumes/Expansion/TRMM_3B42/precipitation/'
data_path = '/Users/ashneelchandra/OneDrive\ -\ The\ University\ of\ Melbourne/Graduate\ Colloquium\ -\ Global\ Hackathon/hackathon-2025-australia-node/Fiji_Meteorological_Service/'


In [3]:
# Define start and end times
start_time = "1998-01-01 00:00:00"
end_time = "2020-01-01 00:00:00"

# Generate the time array with 3-hour intervals
time_array = pd.date_range(start=start_time, end=end_time, freq="3h")

# Convert to a list or leave as a pandas DatetimeIndex
print(time_array)
print(time_array.shape)

DatetimeIndex(['1998-01-01 00:00:00', '1998-01-01 03:00:00',
               '1998-01-01 06:00:00', '1998-01-01 09:00:00',
               '1998-01-01 12:00:00', '1998-01-01 15:00:00',
               '1998-01-01 18:00:00', '1998-01-01 21:00:00',
               '1998-01-02 00:00:00', '1998-01-02 03:00:00',
               ...
               '2019-12-30 21:00:00', '2019-12-31 00:00:00',
               '2019-12-31 03:00:00', '2019-12-31 06:00:00',
               '2019-12-31 09:00:00', '2019-12-31 12:00:00',
               '2019-12-31 15:00:00', '2019-12-31 18:00:00',
               '2019-12-31 21:00:00', '2020-01-01 00:00:00'],
              dtype='datetime64[ns]', length=64281, freq='3h')
(64281,)


In [4]:
#Read test data 
year = 1998
month = 1
day = 1
hour = 0
date_ = datetime(year, month, day, hour)
print(date_)
date_str = date_.strftime("%Y%m%d")
print(date_str)
file = data_path + str(year) + '/' + '3B42.19980101.00.7.HDF'
print(file)

try:
    dataset = SD(file, SDC.READ)
    print(dataset.datasets())
except Exception as e:
    print(f"Error reading file: {e}")

#ds = xr.open_dataset(file, engine="netcdf4")
#ds

1998-01-01 00:00:00
19980101
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.00.7.HDF
{'precipitation': (('nlon', 'nlat'), (1440, 400), 5, 0), 'relativeError': (('nlon', 'nlat'), (1440, 400), 5, 1), 'satPrecipitationSource': (('nlon', 'nlat'), (1440, 400), 22, 2), 'HQprecipitation': (('nlon', 'nlat'), (1440, 400), 5, 3), 'IRprecipitation': (('nlon', 'nlat'), (1440, 400), 5, 4), 'satObservationTime': (('nlon', 'nlat'), (1440, 400), 20, 5), 'InputFileNames': (('fakeDim2',), (0,), 3, 6), 'InputAlgorithmVersions': (('fakeDim3',), (0,), 3, 7), 'InputGenerationDateTimes': (('fakeDim4',), (0,), 3, 8)}


In [5]:
#Create a file list 
pattern = []
files = []
pattern = os.path.join(data_path, f'{year:04d}/3B42.{year:04d}????.??.*.HDF')
files = sorted(glob.glob(pattern))
print(files)
print(len(files))

['/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.00.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.03.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.06.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.09.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.12.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.15.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.18.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.21.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980102.00.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980102.03.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980102.06.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980102.09.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980102.12.7.HDF', '/Volumes/Expansion/TRMM_3B42/precipi

In [7]:
%%time
#Define the lat and lon
theLats = np.arange(-49.875,50,0.25)
theLons = np.arange(-179.875,180,0.25)
print(theLats.shape)
print(theLons.shape)
#print(theLats)
#print(theLons)

# Pattern to extract time information from filenames
time_pattern = r"3B42\.(\d{4})(\d{2})(\d{2})\.(\d{2})"

# Initialize lists
time_list = []  # Initialize list to store timestamps
precip_list = []  # Initialize list for precipitation data 

# Loop through files 
for file in files:
    print(file)
    dataset = []
    precip = []
    timestamp = []
    dataset = SD(file, SDC.READ)
    # List all datasets
    datasets = dataset.datasets()
    precip = dataset.select('precipitation')
    attributes = precip.attributes()
    #print(attributes)
    precip = precip[:]
    #print(precip.shape)
    precip = np.transpose(precip)
    # Set all the missing values less than 0 to NaNs
    np.putmask(precip,precip<0,np.nan)
    print(precip.shape)
    #print(precip)



    # Extract time from the filename
    match = re.match(time_pattern, os.path.basename(file))
    if match:
        year, month, day, hour = map(int, match.groups())
        timestamp = datetime(year, month, day, hour)
        print(timestamp)
        time_list.append(timestamp)

    # Append the data to the list
    precip_list.append(precip)


# Create a single xarray Dataset
data_array = xr.DataArray(
    np.stack(precip_list),  # Stack all data along a new time dimension
    dims=("time", "lat", "lon"),
    coords={"time": time_list, "lat": theLats, "lon": theLons},
    name="precipitation",
    attrs=attributes
)

# Convert to Dataset
ds = xr.Dataset({"precipitation": data_array})

# Shift longitudes from -180 to 180 to 0 to 360
ds = ds.assign_coords(
    lon=((ds.lon + 360) % 360)
)

# Sort the data by longitude to ensure proper ordering
ds = ds.sortby("lon")

print(ds.lon.values)

print(ds)

# Compress and save to netCDF
#print(f'Compressing and saving to netCDF: 3B42.{year:04d}.3hour.nc')
#ds.to_netcdf(path=data_path+f'3B42.{year:04d}.3hour.nc', format="NETCDF4", mode="w", engine="netcdf4", encoding={"precipitation": {"zlib": True, "complevel": 9, "chunksizes": (1, 50, 50)}})
#print(f'Saving of 3B42.{year:04d}.3hour.nc completed successfully')



(400,)
(1440,)
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.00.7.HDF
(400, 1440)
1998-01-01 00:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.03.7.HDF
(400, 1440)
1998-01-01 03:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.06.7.HDF
(400, 1440)
1998-01-01 06:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.09.7.HDF
(400, 1440)
1998-01-01 09:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.12.7.HDF
(400, 1440)
1998-01-01 12:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.15.7.HDF
(400, 1440)
1998-01-01 15:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.18.7.HDF
(400, 1440)
1998-01-01 18:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980101.21.7.HDF
(400, 1440)
1998-01-01 21:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980102.00.7.HDF
(400, 1440)
1998-01-02 00:00:00
/Volumes/Expansion/TRMM_3B42/precipitation/1998/3B42.19980

In [8]:
print(ds.precipitation.shape)
print(ds.precipitation.shape[1])
print(ds.precipitation.shape[2])


(2920, 400, 1440)
400
1440


In [ ]:
precipitation_data = hdf.select('precipitation')

# Get dataset attributes
print("Attributes of 'precipitation' dataset:", precipitation_data.attributes())

precipitation_array = precipitation_data[:]
print(precipitation_array)
print(precipitation_array.shape)

In [ ]:
#Convert to xarray
# Loop through datasets to extract data and attributes
for name, info in datasets.items():
    data = hdf.select(name)[:]  # Load data into NumPy array
    attributes = data.attributes()
    
    # Add to data_vars for xarray
    data_vars[name] = (("lon", "lat"), data)
    
    # You can also include attributes if needed
    data_vars[name] = xr.DataArray(
        data,
        dims=("lon", "lat"),
        attrs=attributes
    )

# Example latitude and longitude (adjust based on file details)
lon = np.linspace(-50, 50, data.shape[0])  # Replace with actual latitudes
lat = np.linspace(-180, 180, data.shape[1])  # Replace with actual longitudes
coords["lon"] = lon
coords["lat"] = lat

# Create xarray Dataset
ds = xr.Dataset(data_vars, coords=coords)

# Inspect the Dataset
print(ds)


In [ ]:

pattern = os.path.join(data_path, 'precipitation_3_hour_????.nc')
files = sorted(glob.glob(pattern))
all_files.extend(files)
print(all_files)
print(all_files[2][43:58])
print(all_files[2][59:71])
print(all_files[2][43:58]+all_files[2][59:71])

In [ ]:
for file in all_files:
    print(file)
    del ds_precip
    ds_precip = xr.open_dataset(file)
    #ds_precip
    #Write netcdf file
    ds_precip.to_netcdf(path='/Volumes/Expansion/TRMM_3B42/'+file[43:58]+file[59:71], mode='w', format="NETCDF4")


In [ ]:
print(all_files[2][43:58]+all_files[2][59:71])